```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Rossby adjustment on different depths

In this notebook we investigate different properties with our numerical schemes related to geostrophic balance when we vary the water depth. Geostrophic balance represent steady state solutions where the pressure gradients are balanced by the Coriolis forces.

## Geostrophic Balance

The geostrophic equations in rotating shalow water is given by
$$ \frac{\partial u}{\partial t} - fv  = - \frac{1}{\rho} \frac{\partial p}{\partial x}, $$
$$ \frac{\partial v}{\partial t} + fu  = - \frac{1}{\rho} \frac{\partial p}{\partial y}. $$
By the assumption that the vertical velocity is negligible compared to the horizontal velocity, we integrate the equations vertically.
Using hydrostatic pressure $ p = \rho g (H+\eta) + p_{atm}$, we get an expression for the change in momentum as
$$ \frac{\partial hu}{\partial t} =  fhv - gh\frac{\partial \eta}{\partial x}, $$
$$ \frac{\partial hv}{\partial t} = -fhu - gh\frac{\partial \eta}{\partial y}. $$
At geostrophic balance, the steady state solution is described by $\frac{\partial hv}{\partial t} = \frac{\partial hu}{\partial t} = 0$.


*What are the references for the above explanation?*

###### Additional geostrophy equation
The following equation is related to the above, to describe the steady-state of the surface elevation:
$$\frac{\partial \eta}{\partial t} + hu\frac{\partial \eta }{\partial x } + hv \frac{\partial \eta }{\partial y} = 0 $$

## Rossby adjustment
Rossby adjustment is the process where an initial bump (of some sort), $\eta_0$, reaches the geostrophic balance steady-state $\bar{\eta}$. The size of the steady-state bump can be described by gravity $g$, lake-at-rest depth $H$ and Coriolis forces $f$ by the Klein-Gordon equation,
$$ -c_0^2 \nabla^2 \bar{\eta} + f^2 (\bar{\eta} - \eta_0) = 0, $$
or, more commonly (?),
$$  \nabla^2 \bar{\eta} -  \left( \frac{1}{a} \right)^2 (\bar{\eta} - \eta_0) = 0. $$
In this equation, $(1/a)^2$ is the Rossby radius. The constant $c_0^2 = gH$, meaning that $a^2 = gH/f^2$. 
The physical interpretation of the Rossby radius, $a$, is shown in the first figure below the initial imports.

Additionally, the steady-state solution should be reached within a period given by $\approx \pi/f$.


## Varying water depth

The height of the steady state bump should change relative to the initial condition bump as follows when the depth is changed.
* When the depth increase, $\bar{\eta}_{max}/{\eta_0}_{max}$ should decrease
* When the depth increase, $\int\bar{\eta}\;/\int{\eta_0}$ should increase.

Here, $\eta_0$ is the initial condition, and $\bar{\eta}$ is the steady state.
The integral is the sum of $\eta$ in all cells in a neighbourhood of the bump.



# Setting up the environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from matplotlib import gridspec as gridspec

import os, pyopencl, datetime, sys

# requires netcdf4-python (netcdf4-python.googlecode.com)
from netCDF4 import Dataset as NetCDFFile

#Finally, import our simulator
from SWESimulators import FBL, CTCS

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Finally, import our simulator
from SWESimulators import FBL, CTCS, KP07, CDKLM16, RecursiveCDKLM16, SimWriter, PlotHelper, Common
from SWESimulators.BathymetryAndICs import *

In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

In [ ]:
#Create output directory for images
imgdir='images_' + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
os.makedirs(imgdir)
print "Saving images to " + imgdir

### Expected results, as explained by Göran in early nov(?) 

In [ ]:
## Qualitatively expected results:
def qualitativExpectedResults():
    fig = plt.figure(figsize=(4, 4))
    depth = np.array(range(100,5100, 500))
    max_height = 2 + np.cos(depth*np.pi/5100)
    integrals = 2 - np.cos(depth*np.pi/5100)
    
    plt.plot(max_height, 'b', label="relative $\eta_{max}$")
    plt.plot(integrals,   'r', label="relative integral$(\eta)$")
    axes = plt.gca()
    axes.set_ylim([0.7, 4])
    plt.legend()
qualitativExpectedResults()


# Setting common parameters

Below are two different sets of parameters. Those from Kai describe a larger (but maybe more realistic) test case, compared to those by Håvard, which are cheaper to run.

In order to activate one or the other, change the cell type of the one you would like to run to "


In [ ]:
# Common parameters from Kai

nx = 800
ny = 1000

dx = 50000
dy = 50000

dt = 100
g = 9.81
f = 1.2e-4
r = 0.0
multi_H0 = np.array(range(100, 5100, 500))*1.0
print(multi_H0)
A = 0.0 # A is diffusion coefficient multiplied by depth.

wind = Common.WindStressParams(type=99)

## Setting up boundary conditions
sponge = [10, 10, 10, 10]
boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=sponge)

ghosts = [10, 10, 10, 10]
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
validDomain =  [10, 10, 10, 10]
cutoff = 10

midx = (nx+2*cutoff)/2
midy = (ny+2*cutoff)/2

# Rossby adjustment parameters:
#c0_squared = g*H0
#a_squared = g*H0/(f*f)

T_steady_state = np.pi/f
print T_steady_state

T = 20
sub_T = 30000.0

interestingDomain = 150

paramSet = "Kai"
msg = "Using variables from " + paramSet

# For plotting:
#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

# Common parameters from Haavard

nx = 200
ny = 200

dx = 200.0
dy = 200.0

dt = 0.09
g = 9.81
f = 0.01
r = 0.0

multi_H0 = np.array(range(30, 150, 20))*1.0
print(multi_H0)
H_step = 0.3

A = 1 # Numerical instability parameter for CTCS

wind = Common.WindStressParams(type=99)

## Setting up boundary conditions
sponge = [10, 10, 10, 10]
boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=sponge)

ghosts = [10, 10, 10, 10]
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
validDomain =  [10, 10, 10, 10]
cutoff = 10

midx = (nx+2*cutoff)/2
midy = (ny+2*cutoff)/2

# Rossby adjustment parameters:
#c0_squared = g*H0
#a_squared = g*H0/(f*f)

T_steady_state = np.pi/f
print T_steady_state

T = 10
sub_T = 100.0

interestingDomain = 30

paramSet = "Haavard"
msg = "Using variables from " + paramSet
# For plotting:
#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

##### Creating initial conditions

Staggered and unstaggered grids are placed so that cell centers are on the same place. Keep therefore in mind that the velocities are defined at different positions for the two different grid types!

In [ ]:
def initialConditions(eta0, nx, ny, dx, dy, halo_x, halo_y):
    if paramSet == "Haavard":
        ghosts = [halo_y, halo_x, halo_y, halo_x]
        addCentralBump(eta0, nx, ny, dx/5, dy/5, ghosts)
        #addCentralDamBreakStep(ctcs_eta0, nx, ny, dx, dy, H_step, ghosts)
        #addCentralDamBreakSmooth(ctcs_eta0, nx, ny, dx, dy, H_step, ghosts)
        return
    print "Making initial conditions"
    x_center = dx*nx/2.0
    y_center = dy*ny/2.0
    y_center1 = dy*(ny-100)/2.0
    y_center2 = dy*(ny+100)/2.0

    for j in range(-halo_y, ny+halo_y):
        for i in range(-halo_x, nx+halo_x):
            x = dx*i - x_center 
            y = dy*j - y_center 
            y1 = dy*j - y_center1
            y2 = dy*j - y_center2

            # Initial smooth step
            inirad = np.sqrt(x**2 + y**2)
            inirad1 = np.sqrt(x**2 + y1**2)
            inirad2 = np.sqrt(x**2 + y2**2)
            L = 15*dx
            D = 50*dx
            etaamp = 0.2
            
            # Add the the initial condition to the provided array
            #eta0[j+1, i+1] += 0.5*etaamp*(1.0+np.tanh((-inirad+D)/L))
            eta0[j, i] += 0.5*etaamp*(1.0+np.tanh((-inirad+D)/L))
            

## Executing simulators

A single test of CTCS to look at the parameters above.

In [ ]:
#Centered in time, centered in space
ctcs_eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
ctcs_u0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32, order='C');
ctcs_v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32, order='C');

ctcs_dt = dt

# Bathymetry:
Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')

H0 = multi_H0[1]

ctcs_h0 = np.ones(dataShape, dtype=np.float32, order='C') * H0;
ctcs_eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
initialConditions(ctcs_eta0, nx, ny, dx, dy, ghosts[1], ghosts[0])

if H0 == 3600:
    ctcs_dt = ctcs_dt/2
if H0 == 4600:
    ctcs_dt = ctcs_dt/2

#Initialize simulator
ctcs_sim = CTCS.CTCS(cl_ctx, \
                     ctcs_h0, ctcs_eta0, ctcs_u0, ctcs_v0, \
                     nx, ny, \
                     dx, dy, ctcs_dt, \
                     g, f, r, A, \
                     wind_stress=wind, \
                     boundary_conditions=boundaryConditions)

fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                ctcs_eta0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                ctcs_u0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                ctcs_v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])

print "CTCS with H0 = " + str(H0) +  " - " + msg


def animate(i):
    if (i>0):
        t = ctcs_sim.step(sub_T)
    else:
        t = 0.0
    eta1, u1, v1 = ctcs_sim.download()
    
    plotter.plot(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]);
    fig.suptitle("CTCS Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%20 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax h: " + str(np.max(eta1)) + \
        "\tMax u: " + str(np.max(u1)) + \
        "\tMax v: " + str(np.max(v1))
        fig.savefig(imgdir + "/{:010.0f}_ctcs.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim


In [ ]:
# Clean up the above simulator
ctcs_h0, ctcs_eta0 = None, None
ctcs_eta1, u1, v1 = None, None, None
if 'ctcs_sim' in globals():
    ctcs_sim.cleanUp()

## Looping over different depth values

In [ ]:
#Coriolis well balanced reconstruction scheme

reload(CDKLM16)

cdklm_relInts = np.zeros_like(multi_H0)
cdklm_relMax = np.zeros_like(multi_H0)
print (multi_H0)
print(cdklm_relInts)
print (cdklm_relMax)

cdklm_u0 = np.zeros(dataShape, dtype=np.float32, order='C');
cdklm_v0 = np.zeros(dataShape, dtype=np.float32, order='C');

# Bathymetry:
Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')
#linearBathymetryY(Bi, nx, ny, dx, dy, ghosts, 5, 50)

for i in range(len(multi_H0)):
    H0 = multi_H0[i]

    cdklm_h0 = np.ones(dataShape, dtype=np.float32, order='C') * H0;
    initialConditions(cdklm_h0, nx, ny, dx, dy, ghosts[1], ghosts[0])
    
    
    #Initialize simulator
    cdklm_sim = CDKLM16.CDKLM16(cl_ctx, \
                                cdklm_h0, cdklm_u0, cdklm_v0, \
                                Bi, \
                                nx, ny, \
                                dx, dy, dt, \
                                g, f, r, \
                                wind_stress=wind, \
                                boundary_conditions=boundaryConditions, \
                                reportGeostrophicEquilibrium=True)

    #Kx = np.zeros(T)
    #Ly = np.zeros(T)
    #uxpvy = np.zeros(T)
    fig = plt.figure()
    plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                    cdklm_h0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - H0, 
                                    cdklm_u0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                    cdklm_v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])

    print "CDKLM with H0 = " + str(H0) +  " - " + msg
    for tLoop in range(T):
        t = cdklm_sim.step(sub_T)
        h1, u1, v1 = cdklm_sim.download()
        #uxpvy[i], Kx[i], Ly[i] = cdklm_sim.downloadGeoEqNorm()
        
        plotter.plot(h1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - H0, 
                     u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                     v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]);
        fig.suptitle("CDKLM16 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

        print "{:03.0f}".format(100.0*tLoop / T) + " % => t=" + str(t) + "\tMax h: " + str(np.max(h1)) + \
        "\tMax u: " + str(np.max(u1)) + "\tMax v: " + str(np.max(v1))
        
        #if (i%10 == 0):
        fig.savefig(imgdir + "/H0{:05.0f}_H0".format(H0) + "_{:010.0f}_cdklm16.png".format(t))


    print("Finished CDKLM with H0 = " + str(H0) + ", simulating until t = " + str(cdklm_sim.t))
    
    # Computing the interesting values:    
    h1, u1, v1 = cdklm_sim.download()
    cdklm_eta0 = cdklm_h0[midy-interestingDomain:midy+interestingDomain,\
                          midx-interestingDomain:midx+interestingDomain] - H0
    cdklm_eta = h1[midy-interestingDomain:midy+interestingDomain,\
                   midx-interestingDomain:midx+interestingDomain] - H0    
    
    #print "sum(sum(cdklm_eta))", type(sum(sum(cdklm_eta)))
    print "sum(sum(cdklm_eta))", sum(sum(cdklm_eta))
    print "sum(sum(cdklm_eta0))", sum(sum(cdklm_eta0))
    cdklm_relInts[i] = sum(sum(cdklm_eta)) / sum(sum(cdklm_eta0))
    cdklm_relMax[i] = np.max(cdklm_eta) / np.max(cdklm_eta0)
    
    fig.clear()
    fig = plt.figure(figsize=(4,4))
    plt.imshow(cdklm_eta)
    plt.colorbar()

    cdklm_h0 = None
    cdklm_eta, cdklm_eta0 = None, None
    h1, u1, v1 = None, None, None
    if 'cdklm_sim' in globals():
        cdklm_sim.cleanUp()

In [ ]:
# Make nice Göran plots!
print multi_H0.shape, cdklm_relInts.shape, cdklm_relMax.shape

fig = plt.figure(figsize=(6,4))
plt.plot(multi_H0, cdklm_relInts)
plt.title("relative integrals")
plt.xlabel("depth")

fig = plt.figure(figsize=(6,4))
plt.plot(multi_H0, cdklm_relMax)
plt.title("relative max height")
plt.xlabel("depth")

#= sum(sum(cdklm_eta)) / sum(sum(cdklm_eta0))
#    cdklm_relMax[i])

In [ ]:
#Centered-in-Time, Centered-in-space

reload(CTCS)

ctcs_relInts = np.zeros_like(multi_H0)
ctcs_relMax = np.zeros_like(multi_H0)
print (multi_H0)
print(ctcs_relInts)
print (ctcs_relMax)


ctcs_eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
ctcs_u0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32, order='C');
ctcs_v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32, order='C');

ctcs_dt = dt

# Bathymetry:
Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')
#linearBathymetryY(Bi, nx, ny, dx, dy, ghosts, 5, 50)

for i in range(len(multi_H0)):
    H0 = multi_H0[i]

    ctcs_h0 = np.ones(dataShape, dtype=np.float32, order='C') * H0;
    ctcs_eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
    initialConditions(ctcs_eta0, nx, ny, dx, dy, ghosts[1], ghosts[0])
    
    if H0 == 3600:
        ctcs_dt = ctcs_dt/2
    if H0 == 4600:
        ctcs_dt = ctcs_dt/2
    
    #Initialize simulator
    ctcs_sim = CTCS.CTCS(cl_ctx, \
                         ctcs_h0, ctcs_eta0, ctcs_u0, ctcs_v0, \
                         nx, ny, \
                         dx, dy, ctcs_dt, \
                         g, f, r, A, \
                         wind_stress=wind, \
                         boundary_conditions=boundaryConditions)

    fig = plt.figure()
    plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                    ctcs_eta0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                    ctcs_u0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                    ctcs_v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])

    print "CTCS with H0 = " + str(H0) +  " - " + msg
    for tLoop in range(T):
        t = ctcs_sim.step(sub_T)
        eta1, u1, v1 = ctcs_sim.download()
        
        plotter.plot(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                     u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                     v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]);
        fig.suptitle("CTCS Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

        print "{:03.0f}".format(100.0*tLoop / T) + " % => t=" + str(t) + "\tMax h: " + str(np.max(eta1)) + \
        "\tMax u: " + str(np.max(u1)) + "\tMax v: " + str(np.max(v1))
        
        #if (i%10 == 0):
        fig.savefig(imgdir + "/H0{:05.0f}_H0".format(H0) + "_{:010.0f}_ctcs.png".format(t))


    print("Finished CTCS with H0 = " + str(H0) + ", simulating until t = " + str(ctcs_sim.t))
    
    # Computing the interesting values:    
    ctcs_eta1, u1, v1 = ctcs_sim.download()
    
    #print "sum(sum(ctcs_eta))", type(sum(sum(ctcs_eta)))
    print "sum(sum(ctcs_eta1))", sum(sum(ctcs_eta1))
    print "sum(sum(ctcs_eta0))", sum(sum(ctcs_eta0))
    ctcs_relInts[i] = sum(sum(ctcs_eta1)) / sum(sum(ctcs_eta0))
    ctcs_relMax[i] = np.max(ctcs_eta1) / np.max(ctcs_eta0)
    
    fig.clear()
    fig = plt.figure(figsize=(4,4))
    plt.imshow(ctcs_eta1)
    plt.colorbar()
    
    ctcs_h0, ctcs_eta0 = None, None
    ctcs_eta1, u1, v1 = None, None, None
    if 'ctcs_sim' in globals():
        ctcs_sim.cleanUp()

In [ ]:
# Make nice Göran plots!
print multi_H0.shape, ctcs_relInts.shape, ctcs_relMax.shape

fig = plt.figure(figsize=(6,4))
plt.plot(multi_H0, ctcs_relInts)
plt.title("CTCS - relative integrals")
plt.xlabel("depth")

fig = plt.figure(figsize=(6,4))
plt.plot(multi_H0, ctcs_relMax)
plt.title("CTCS - relative max height")
plt.xlabel("depth")

#= sum(sum(cdklm_eta)) / sum(sum(cdklm_eta0))
#    cdklm_relMax[i])